## Initial Setup

1. Download the Zeus 2 version 2.0.8.9 reference sample from [MalwareBazaar](
https://bazaar.abuse.ch/sample/0fd0fedc1c314629e83f3e2ef6867e5cbaf15c4bd6235a8d9fadfc79bd441611/).
2. Unpack the sample with [UNPACME](https://www.unpac.me/results/0043ed23-cdd7-47a5-8ee6-f26f248cbdb7#/).
3. Zeus 2 2.0.8.9 [source code](https://github.com/Visgean/Zeus).

In [1]:
# Install required Python modules
import sys
!{sys.executable} -m pip install pefile

# Import required Python modules
import pprint
import re
import struct

import pefile

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Load the unpacked sample
fp = open("d146fe1e7b782cfd1c534c44c63ba578df0acd184cab86d0e5490eb39108c84e.bin", "rb")
sample = fp.read()
fp.close()

pe = pefile.PE(data=sample)

## Decrypt Strings

Pseudocode:

    int __usercall CryptedStrings::_getA@<eax>(unsigned __int16 id@<ax>, int buffer@<edi>)
    {
      STRINGINFO *s; // eax
      int i; // ecx
      int i2; // esi
      char pb; // dl
      int size; // eax

      s = &dword_4025F8[2 * id];
      for ( i = 0; i < *&s->size; *(i2 + buffer) = pb )
      {
        i2 = i;
        pb = i ^ s->key ^ *(*&s->encodedString + i);
        ++i;
      }
      size = *&s->size;
      *(size + buffer) = 0;
      return size;
    }

Disassembly:

    .text:0040F314                               CryptedStrings___getA proc near
    .text:0040F314 0F B7 C0                                      movzx   eax, ax
    .text:0040F317 8D 04 C5 F8 25 40 00                          lea     eax, dword_4025F8[eax*8]
    .text:0040F31E 33 D2                                         xor     edx, edx
    .text:0040F320 33 C9                                         xor     ecx, ecx
    .text:0040F322 66 3B 50 02                                   cmp     dx, [eax+2]
    .text:0040F326 73 19                                         jnb     short loc_40F341
    .text:0040F328 56                                            push    esi
    .text:0040F329
    .text:0040F329                               loc_40F329:
    .text:0040F329 8B 50 04                                      mov     edx, [eax+4]
    .text:0040F32C 0F B7 F1                                      movzx   esi, cx
    .text:0040F32F 8A 14 32                                      mov     dl, [edx+esi]
    .text:0040F332 32 10                                         xor     dl, [eax]
    .text:0040F334 32 D1                                         xor     dl, cl
    .text:0040F336 41                                            inc     ecx
    .text:0040F337 88 14 3E                                      mov     [esi+edi], dl
    .text:0040F33A 66 3B 48 02                                   cmp     cx, [eax+2]
    .text:0040F33E 72 E9                                         jb      short loc_40F329
    .text:0040F340 5E                                            pop     esi
    .text:0040F341
    .text:0040F341                               loc_40F341:
    .text:0040F341 0F B7 40 02                                   movzx   eax, word ptr [eax+2]
    .text:0040F345 C6 04 38 00                                   mov     byte ptr [eax+edi], 0
    .text:0040F349 C3                                            retn
    .text:0040F349                               CryptedStrings___getA endp

In [3]:
def decrypt_string(pe, id_):
    match = re.search(rb'\x8d\x04\xc5(?P<enc_strings_addr>[\s\S]{4})', 
                      pe.get_memory_mapped_image())
    enc_strings_addr = struct.unpack("I", match.group("enc_strings_addr"))[0]
    
    enc_string_addr = enc_strings_addr + 8 * id_
    enc_string_rva = enc_string_addr - pe.OPTIONAL_HEADER.ImageBase
    
    xor_key = pe.get_word_at_rva(enc_string_rva)
    encbuf_len = pe.get_word_at_rva(enc_string_rva + 2)
    encbuf_addr = pe.get_dword_at_rva(enc_string_rva + 4)
    encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
    encbuf = pe.get_data(encbuf_rva, encbuf_len)
    
    plainbuf = b""
    for i in range(len(encbuf)):
        pb = (i ^ xor_key ^ encbuf[i]) & 0xff
        plainbuf += struct.pack("B", pb)
        
    return plainbuf.decode()


for i in range(230):
    plain_string = decrypt_string(pe, i)
    print(f"{i}: {plain_string}")

0: screenshots\%s\%04x_%08x.jpg
1: unknown
2: image/jpeg
3: Software\Microsoft\Windows\Currentversion\Run
4: SOFTWARE\Microsoft\Windows NT\CurrentVersion\ProfileList\%s
5: ProfileImagePath
6: unknown\unknown
7: :d
rd /S /Q "%s"
rd /S /Q "%s"
if exist "%s" goto d
if exist "%s" goto d
8: grabbed\%S_%02u_%02u_%02u.txt
9: Grabbed data from: %s

%S
10: %s%s
Referer: %S
User input: %s
%sPOST data:

%S
11: *EMPTY*
12: *UNKNOWN*
13:  *BLOCKED*
14: Content-Type: %s

15: ZCID: %S

16: application/x-www-form-urlencoded
17: HTTP authentication: username="%s", password="%s"

18: HTTP authentication (encoded): %S

19: %s://%s:%s@%s/
20: ftp
21: pop3
22: anonymous
23: Software\Microsoft\Internet Explorer\Main
24: Start Page
25: Software\Microsoft\Internet Explorer\PhishingFilter
26: Enabled
27: EnabledV8
28: Software\Microsoft\Internet Explorer\Privacy
29: CleanCookies
30: Software\Microsoft\Windows\CurrentVersion\Internet Settings\Zones\%u
31: 1406
32: 1609
33: Accept-Encoding: identity

34: TE:

35

## Extract Version

Pseudocode:

    ...
    version = 0x2000809;
    r = BinStorage::_addItem(binStorage, 4u, 10003, 0x20000, &version);// #define SBCID_BOT_VERSION 10003
    ...
    
Disassembly:

    ...
    .text:00417E64 8D 45 F8                      lea     eax, [ebp+version]
    .text:00417E67 50                            push    eax
    .text:00417E68 57                            push    edi
    .text:00417E69 68 13 27 00 00                push    2713h
    .text:00417E6E 6A 04                         push    4
    .text:00417E70 5B                            pop     ebx
    .text:00417E71 8B C6                         mov     eax, esi
    .text:00417E73 C7 45 F8 09 08 00 02          mov     [ebp+version], 2000809h
    .text:00417E7A E8 68 2C FF FF                call    BinStorage___addItem
    ...

In [4]:
match = re.search(rb'\x68\x13\x27\x00\x00[\s\S]{1,8}?\xc7\x45[\s\S](?P<version>[\s\S]{4})', 
                  pe.get_memory_mapped_image())
version_bytes = match.group("version")
version = ".".join([str(b) for b in version_bytes[::-1]])
print(version)

2.0.8.9


## Decrypt BaseConfig

Pseudocode:

    int __usercall Core::getBaseConfig@<eax>(void *bc@<eax>)
    {
      int result; // eax
      int v2; // edx
      int v3; // esi
      int v4; // ecx

      Mem::_copy(bc, dword_402D88, 0x32Cu);
      v3 = v2;
      v4 = dword_423E70 + dword_4239C4 - result;
      do
      {
        *result ^= *(v4 + result);
        ++result;
        --v3;
      }
      while ( v3 );
      return result;
    }

Disassembly:

    .text:0041CC1D                               Core__getBaseConfig proc near
    .text:0041CC1D 56                                            push    esi
    .text:0041CC1E BA 2C 03 00 00                                mov     edx, 32Ch
    .text:0041CC23 52                                            push    edx
    .text:0041CC24 68 88 2D 40 00                                push    offset dword_402D88
    .text:0041CC29 50                                            push    eax
    .text:0041CC2A E8 F3 85 FE FF                                call    Mem___copy
    .text:0041CC2F 8B 0D C4 39 42 00                             mov     ecx, dword_4239C4
    .text:0041CC35 03 0D 70 3E 42 00                             add     ecx, dword_423E70
    .text:0041CC3B 8B F2                                         mov     esi, edx
    .text:0041CC3D 2B C8                                         sub     ecx, eax
    .text:0041CC3F
    .text:0041CC3F                               loc_41CC3F:         
    .text:0041CC3F 8A 14 01                                      mov     dl, [ecx+eax]
    .text:0041CC42 30 10                                         xor     [eax], dl
    .text:0041CC44 40                                            inc     eax
    .text:0041CC45 4E                                            dec     esi
    .text:0041CC46 75 F7                                         jnz     short loc_41CC3F
    .text:0041CC48 5E                                            pop     esi
    .text:0041CC49 C3                                            retn
    .text:0041CC49                               Core__getBaseConfig endp

In [5]:
match = re.search(rb'\xba(?P<encbuf_len>[\s\S]{2}\x00\x00)\x52\x68(?P<encbuf_addr>[\s\S]{4})',
                   pe.get_memory_mapped_image())

encbuf_len = struct.unpack("I", match.group("encbuf_len"))[0]
encbuf_addr = struct.unpack("I", match.group("encbuf_addr"))[0]
encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
encbuf = pe.get_data(encbuf_rva, encbuf_len)

# #define PESECTION_OF_BASECONFIG_KEY 2
section = pe.sections[2]
xor_key = pe.get_data(section.VirtualAddress, encbuf_len)

base_config = b""
for i, eb in enumerate(encbuf):
    pb = eb ^ xor_key[i % len(xor_key)]
    base_config += struct.pack("B", pb)
    
print(base_config)

b'w\x90\xdb\xe7\xeb\x91\xf8m\x07{\xc6\xa2\xc4n\xb1\xd9}i\x8c!S\x1a\x1e\xbd\xd0\xf4\xdd\xd4\xcb\x83\xf9j\x8b\xfc\x9c\x8f/\x19q\xd6L#c\t\xd9P\xa7\x1bHP\xfc^\xf6#1\xf6\xc5-\xe8\x8e\x07\x8c\x9bJ\x9f\xa9\xdc\xfcb\r\xe1\x9a\xc4w?\xf0\xb0+\xf6\x92\x00\x00\x00 m\xed@~\x10\xa5x\xfbka\x1a\xd1\xfa\xbf"\x96O\x18P\xa0\xb4\x9c\xe4\xbd\x12\xa5\xa7$\x12E*\xac\x81!R<\xf9\x83\x14\x83\xbe\xaa~\xe9\xe6e\xf7\xc5\xbd\xe0\xc7\xd0g\xbd\xda\xda\xc7f\xc0\xbb\x01\x00\x14\x00\xfagp\x18\xc8\xf0E\x8e\xc7\x0e\xcfy\xa9\xc3\xe2\xf1s\rM\x12$\x00\xa7\xad\x97\x17L>\xaa\xcc\xa2\x1e\xa0f\xa1\xfa\xc4\x13\x92\x84O6\x16h\x8c\x02k\x82:H\xebN2\x8d\xd5\xa0J`P\x95x\xeb\xad\xc9;\xc7\xed\xca6\x15\x83\xcd\x1e\xca[\xa0\xd7J\x1a\xe0\xda9\xa8l\xda\x8d\xec\x19\xa7\xe8\xd6http://emailupgrade.su/new/config.bin\x00M\xf7\xd5\x9cTr\xf9\xb1\xd3\xfb\x88\xc5\x93\x0c\xa4\xc5\xda\x9f\xef\x18\x85\xf1B\x82W\x86\xa2I\x91mo\x97\xc3\xc5\xdf \xe1\x80\xab\x04\x99g\x08\xf7\xe4^\xd9\x9f\xdc\xea#=\xcd\xf0\xc1\x90\x8e\xc0Y\xf7\x12Q\x8e\xd5:\xcew9\xedR%\xae\

## Parse BaseConfig

In [6]:
flags = struct.unpack("I", base_config[0x50:0x50+4])[0]
print(f"flags: 0x{flags:x}\n")

delayStats = struct.unpack("I", base_config[0x90:0x90+4])[0]
print(f"delayStats: {delayStats}\n")

defaultConfig = base_config[0xef:0xef+101].split(b"\x00")[0].decode().replace("http", "hxxp")
print(f"defaultConfig: {defaultConfig}\n")

baseKey = base_config[0x15b:0x15b+258]
print(f"baseKey: {baseKey}\n")

defaultBotnet = base_config[0x2aa:0x2aa+21].split(b"\x00\x00")[0].decode()
print(f"defaultBotnet: {defaultBotnet}\n")

delayReport = struct.unpack("I", base_config[0x2fc:0x2fc+4])[0]
print(f"delayReport: {delayReport}\n")

delayConfig = struct.unpack("I", base_config[0x310:0x310+4])[0]
print(f"delayConfig: {delayConfig}\n")

flags: 0x20000000

delayStats: 1310721

defaultConfig: hxxp://emailupgrade.su/new/config.bin

baseKey: b'%\xae\x19\xee\xf2^-4\xaat]\x95\x8enz9\xd6T\xc8<\xbe \x03\xe06\xa7\xb7\xb23\x15(\x1aK\'Bx@+\x05D.\xfb~\xbc\x1e\x9eQ\xc0\x17\xb8\xef[H\xc7\x82o/h\xe7N\xa6V:\x7f\xe1\x86I\xed\x9f\xe5\x01\xf9X`}\xcf\xf8\x8d\xbd\xcd\x8b\x8c\xbf\xb5s\x1d\xadb\xf5\x84wy&v\x10f\xfe8P\x02\x81\x97\xf3\xf0\xceu\xd2\x99\x9dOE\x1c\xa1\x9b\xa5\xd05\x96\xd1W>)e?\x12\x880\xccR\xdd=!_\xab\xc2\xe8\xa9\x0b\x94\x98Y\x80G\x1f\x92\xdb\xca\x90"\xa8\x9aJ\xc3L\xe2\x8fF\xfd\x04\xde\xe3\xf1\x07\x0c\xeb\xda\x0f\x83\xec\xe4\xa3#\xc9\xa0\x85\xc5\x1b\xd9\x8a\xfc\xa2\xd7\x06\x93\n\xf7A\x18\r\x00;\x9c\xcb\x11j\x08\xbal\xd4\xb3\xd5q7Z\xb6ra\tc\xc1M\xdfi2m\x14\xb1,\xf4\xdc*\xe9g1U\x16\x13\xafC\xac\xb9\xd3\xeap\xb0\xbbS\x0e\xfak\x91\xd8{\xa4\xff\xc4\xf6\xe6\xc6\\\x89\xb4\x87$|d\x00\x00'

defaultBotnet: 

delayReport: 65537

delayConfig: 3932161



## Download and Decrypt DynamicConfig

![](zeus2_2.0.8.9_pcap.png)

Pseudocode:

    ...
    Mem::_copy(v7, rc4Key, 0x102u);
    Crypt::_rc4(v4, binStorage, dataSize);
    if ( dataSize )                             // inlined Crypt::_visualDecrypt
    {
      v5 = dataSize - 1;
      if ( dataSize != 1 )
      {
        do
        {
          *(v5 + binStorage) ^= *(v5 + binStorage - 1);
          --v5;
        }
        while ( v5 );
      }
    }

In [7]:
def rc4_keystate(S, in_buf):
    out_buf = b""

    i = S[256]
    j = S[257]
    
    for in_byte in in_buf:
        (i, j, S, K) = rc4_prga(i, j, S)
        out_byte = in_byte ^ K
        out_buf += struct.pack("B", out_byte)

    return out_buf


def rc4_prga(i, j, S):
    i = (i + 1) % 256
    j = (j + S[i]) % 256

    S = bytearray(S)
    S[i], S[j] = S[j], S[i]
    S = bytes(S)

    K = S[(S[i] + S[j]) % 256]

    return (i, j, S, K)


def visual_decrypt(enc_buf):
    buf = bytearray(enc_buf)

    for i in range(len(buf)-1, 0, -1):
        buf[i] ^= buf[i-1]

    plain_buf = bytes(buf)

    return plain_buf


# Load encrypted DynamicConfig from PCAP
fp = open("config.bin", "rb")
config_bin = fp.read()
fp.close()

round1 = rc4_keystate(baseKey, config_bin)
binstorage = visual_decrypt(round1)
print(binstorage)

b'6q\x1fT\xe1\x1b\x91\xa0\x0f!\xcb\x91\xce\xcf\x8b\xc4X\xfa\xe8\x97l\x86\x00\x00\x00\x00\x00\x00u\x00\x00\x00\xb5<\xa7H\xd7%\x97)M\xa23\x1d\x9dK\xa8\xe5!N\x00\x00\x00\x00\x00\x10\x04\x00\x00\x00\x04\x00\x00\x00\t\x08\x00\x02"N\x00\x00\x00\x00\x00\x10"\x00\x00\x00"\x00\x00\x00http://emailupgrade.su/new/bot.exe#N\x00\x00\x00\x00\x00\x10#\x00\x00\x00#\x00\x00\x00http://emailupgrade.su/new/gate.php%N\x00\x00\x01\x00\x00\x10\x85\x00\x00\x00\xa1\x00\x00\x00\xdb\xff\xff\xff!*.microsoft.com/*\x00!http:/\tm\xd6}\xed\xfdyspace\x16\x15\x14s\x15w\x00\xb0\xbf\xbd\xfd.grup1ant\x02der.es5\xc0\xda\xff?odnoklassniki.)\x1b\xfb?\xbb{vko3kte\x16@*/login.K0`Ovmp\x12atl\x10\x00\x00\x00\x00\x00\x00\x00H\x00\xff\x01\x00\x00\x00\x00\x00\x00@>\x00\x00\x00>\x00\x00\x00)\x00\x00\x00 Registered email address</td>*<img*>\t\x00\x00\x00</td>\x0c\x00\x00\x00e-mail: \x02\x00\x00\x00\x01\x00\x00@^\x00\x00\x00^\x00\x00\x00\xff\xff\xff\xcbH\x00(<a href="http://feedback\xff\xdb\xff\xdb.e\x05y.com/ws/eB\x0bISAPI.dll}\xf7\x8

## Parse DynamicConfig

In [8]:
def parse_binstorage_header(binstorage):
    header = {
        "randData": binstorage[0:20],
        "size": struct.unpack("I", binstorage[20:24])[0],
        "flags": struct.unpack("I", binstorage[24:28])[0],
        "count": struct.unpack("I", binstorage[28:32])[0],
        "md5Hash": binstorage[32:48],
    }
        
    return header


def parse_binstorage_sections(section_data, num_sections):
    CFGIDS = {
        20001: "CFGID_LAST_VERSION",
        20002: "CFGID_LAST_VERSION_URL",
        20003: "CFGID_URL_SERVER_0",
        20004: "CFGID_URL_ADV_SERVERS",
        20005: "CFGID_HTTP_FILTER",
        20006: "CFGID_HTTP_POSTDATA_FILTER",
        20007: "CFGID_HTTP_INJECTS_LIST",
        20008: "CFGID_DNS_LIST",
    }
    
    sections = []
    offset = 0
    for i in range(num_sections):
        section = {}
        
        id_ = struct.unpack("I", section_data[offset:offset+4])[0]
        if int(id_) in CFGIDS:
            section["id"] = f'{CFGIDS[int(id_)]} ({id_})'
        else:
            section["id"] = id_
            
        section["flags"] = struct.unpack("I", section_data[offset+4:offset+8])[0]
        section["size"] = struct.unpack("I", section_data[offset+8:offset+12])[0]
        section["realSize"] = struct.unpack("I", section_data[offset+12:offset+16])[0]
        # additional parsing of data is left as an exercise
        section["data"] = section_data[offset+16:offset+16+section["size"]]
        
        offset += 4 + 4 + 4 + 4 + section["size"]
        
        sections.append(section)
        
    return sections


binstorage_header = parse_binstorage_header(binstorage)
print("binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(binstorage[48:], binstorage_header["count"])
print()
print("binstorage_sections:\n")
pprint.pprint(binstorage_sections)

binstorage_header:

{'count': 117,
 'flags': 0,
 'md5Hash': b'\xb5<\xa7H\xd7%\x97)M\xa23\x1d\x9dK\xa8\xe5',
 'randData': b'6q\x1fT\xe1\x1b\x91\xa0\x0f!\xcb\x91\xce\xcf\x8b\xc4'
             b'X\xfa\xe8\x97',
 'size': 34412}

binstorage_sections:

[{'data': b'\t\x08\x00\x02',
  'flags': 268435456,
  'id': 'CFGID_LAST_VERSION (20001)',
  'realSize': 4,
  'size': 4},
 {'data': b'http://emailupgrade.su/new/bot.exe',
  'flags': 268435456,
  'id': 'CFGID_LAST_VERSION_URL (20002)',
  'realSize': 34,
  'size': 34},
 {'data': b'http://emailupgrade.su/new/gate.php',
  'flags': 268435456,
  'id': 'CFGID_URL_SERVER_0 (20003)',
  'realSize': 35,
  'size': 35},
 {'data': b'\xdb\xff\xff\xff!*.microsoft.com/*\x00!http:/\tm\xd6}\xed\xfdyspa'
          b'ce\x16\x15\x14s\x15w\x00\xb0\xbf\xbd\xfd.grup1ant\x02der.e'
          b's5\xc0\xda\xff?odnoklassniki.)\x1b\xfb?\xbb{vko3kte\x16@*/login.K'
          b'0`Ovmp\x12atl\x10\x00\x00\x00\x00\x00\x00\x00H\x00\xff',
  'flags': 268435457,
  'id': 'CFGID_HTTP_FIL

  'realSize': 357,
  'size': 297},
 {'data': b'\xfb\xff\xff\xcb\x19\x00name="Password"*</tr>\x04\x18\xfd\xed\xb1'
          b'\xb6\xe2\x01\x03\x0c\r\n<\x05 align\xad\xfdc\xbf(left"\x12d bgco'
          b'l4\x14`\'\xec\xed#ce\x16e7"$ v\r\xe6\xfeX\xfbmidd\x114div clk\x15'
          b"n?\xf6\xedenTies3\x17b>MemIab\xe6\xbe\xed\xdb' \x86:xb\x17/"
          b'0\x07\xfd\x1fa-ms><img srcm\xfb\xf6\xae\x12\x08/spa\x18.gif'
          b'\x7fww\xd6\xdd\xde\xdath\x15a he\x88ht\n0\x0bKdw\xddb^e?0">V'
          b'\xad\xc3\xffX\x02\x16input typ7[kmHp\x0f\xbc\x18Yx\xed=\xa1m\x132'
          b'\x0cTUK\x1e\r\xbd\xdb\xdb\xe6siz15\tmax\xe1ng\x96~\x08\x0b\xcc\x0e'
          b'\xe16top"\xe4\n\xec\xc3\x97 = \xf01\xdb\xc3\x02\xb94\xd5 \x00'
          b'\x17\x85d\xc3\x0c\xe5\xcd\xbd.n\xb7w\x0f\xfc\x86\x03 OnC\x8ack\x13'
          b'ja\xbf\xb5/\xdc\x94sc\x00pt: h8(docu\xb3\xbf]h\xe1\x13.f\\ms[0].'
          b"\xeb\xda\x7f\xd6\xba.\xbcu\x8d\xe3 < 2) {o\x15Z\xfb\xd6\x13\x7ft('P"
          b'\x06"\xdf\x13\xc2\xf6\xc

 {'data': b'\xeb\xfe\xff\xcb\x13\x00NAME="PAN"*<TR>\x04\x12\xb8\x03\xff\xbf\xfd'
          b'\x9b\x0bD ALIGN\x1bCENTER" CLASS\xff\xff\xffo\x0epx12"><B>Firma e'
          b'lectronica.N\xf7\xd6\xda</\x15\x039>\r\n\x06R\xff\xde\xca'
          b'\xee\x05 \r\nM@\x18INPU\xef\xde\xfe\xd9T \x84ESpasshTYP\rte'
          b'xt\xf6\xed\xd6n\x0bid\t/XT\tsize\x0b\xec}\xdb\xda8\x08txsngth'
          b'\rj\xdb\xff7Xu%\xceumPulsaci\x92e\xef\xbe_\xd8s=\x0e-1*;\xe0'
          b'\xa4\x03if(dwo\xdf\xdaoc\x1eIt.fo\xc9s.n\x05al.\xfd\xb3\xb7g\x85.'
          b'v\nue.h < 5){\xd6\xee\xfe\xadi\x12ert("In\xf0duz\xf1\x94d\xed)\xe1u'
          b' \x06");*\x87\xed&271\x12nf\x1cs(\x00\x00\xb7\xf69}\x02Lse'
          b'\x04\x00\x00\x00\x00\x00\x00\x80\xff',
  'flags': 1073741825,
  'id': 54,
  'realSize': 412,
  'size': 301},
 {'data': b'\xff\xed\xff\xcb#\x00/Jaen/images\x06nterroga\xffm\xf7w\x07.jpg'
          b'*>\x04"\x02\x01\x03</TR> \r\n\x1f\xfb\xff\xb0<\x06D height=29'
          b'\x10P><F\xfb\xb7\xed\xffONT fa

  'size': 335},
 {'data': b']\xf7\xff\xcb\x10\x00name="PIN"*>\x04\x0f\x81\x03<"\x9b\xeb\xfe/'
          b'tr>\r\n<\x05 \x06\rd\x01\xfb\xed\xdf\x05Clave d\x02Firm'
          b'a\x18\xb3\xdf\xfd\x7f&input typOtext" [\xb7\xd6\xbc\xfdESpass\rs'
          b'iz8\x089xl\xc1v\x1f\xfbength\rvalu\x17">im\xff\xff\xbb!\x80funct'
          b'ion Enviar_fonu\xf6\xef\xbe\xbf{ri()*{\xa5w\x03if (docuc\x8d\x9d'
          b"}\xffnt.userf.m.U.\xbe\xb5\xf6\x9ff < 4) {_\x13$t(' \xd6\xb6"
          b"\xdb\xc2\xc6\xa6o 'cm\xee\xed\xf7\xdb\xdbaO');(bPotE\x0fr="
          b'\x04\x00\xb4\xadkG\x11}"lb\x00\x00\x00\x00\x00\x00\x00\x12\xff',
  'flags': 1073741825,
  'id': 64,
  'realSize': 305,
  'size': 252},
 {'data': b'\xf7\xff\xff\xcb\x18\x00type="password"*<tr>\x04\x17\xb7\xed\xdf\\'
          b'\xef\x03\x0bd align r\x05hto\xff\x1f\xec" v\x0ecenter">\r\n<s;'
          b'n c\xfd\xef\xad\xb9l@\x16\x05bel\x15Firma \xed\xeb~\xbb\x0becNoni'
          b'ca(/)>\x06\xff\x01\xd8kT6Zinput i\xf6mk\xb3d\x9f"RxQngth\x10'
 

 {'data': b'\x14\x00\x00\x00<h1>log on</h1> \x0b\x00\x00\x00</html>\x04'
          b'\x00\x00\x00',
  'flags': 1073741824,
  'id': 91,
  'realSize': 35,
  'size': 35},
 {'data': b'\xff\xff\xff\xcb\x1c\x00<fieldset class="login">V\xffo\xff\x7f\x1b'
          b'table width\x18100%" borde\xdb~\xdb\xb6r\r0\nc8lspac.g\x0f\x9f%\xbb'
          b'%\x0edd\x0fPsub\x00\x80\xd9\xfemit-barU\x04\x01\x00\x00\x00\x00\x00'
          b'\x00 \xff',
  'flags': 1073741825,
  'id': 92,
  'realSize': 118,
  'size': 111},
 {'data': b'\x16\x00\x00\x00<span class="sum">\x0b\x00\x00\x00</span'
          b'>\x04\x00\x00\x00',
  'flags': 1073741824,
  'id': 93,
  'realSize': 37,
  'size': 37},
 {'data': b'\x16\x00\x00\x00<span class="sum">\x0b\x00\x00\x00</span'
          b'>\x04\x00\x00\x00',
  'flags': 1073741824,
  'id': 94,
  'realSize': 37,
  'size': 37},
 {'data': b'\x19\x00\x00\x00<span id="money_sum">\x0b\x00\x00\x00</span>'
          b'\x04\x00\x00\x00',
  'flags': 1073741824,
  'id': 95,
  'realSize': 40

## Commands

Pseudocode:

    ...
    ci = 0;
    while ( 1 )
    {
        CryptedStrings::_getW(word_402530[4 * ci], commandNameBuffer);
        if ( !lstrcmpiW(*args, commandNameBuffer) )
            break;
        if ( ++ci >= 25 )
            goto LABEL_33;
    }
    ...
    
Disassembly:

    ...
    .text:004193FF 33 FF                                         xor     edi, edi
    .text:00419401
    .text:00419401                               loc_419401:
    .text:00419401 66 8B 04 FD 30 25 40 00                       mov     ax, ds:word_402530[edi*8]
    .text:00419409 8D B5 54 FD FF FF                             lea     esi, [ebp+commandNameBuffer]
    .text:0041940F E8 36 5F FF FF                                call    CryptedStrings___getW
    .text:00419414 8B C6                                         mov     eax, esi
    .text:00419416 50                                            push    eax
    .text:00419417 FF 33                                         push    dword ptr [ebx]
    .text:00419419 FF 15 F0 10 40 00                             call    ds:lstrcmpiW
    .text:0041941F 85 C0                                         test    eax, eax
    .text:00419421 74 08                                         jz      short loc_41942B
    .text:00419423 47                                            inc     edi
    .text:00419424 83 FF 19                                      cmp     edi, 19h
    .text:00419427 72 D8                                         jb      short loc_419401
    .text:00419429 EB 1F                                         jmp     short loc_41944A
    ...

In [9]:
match = re.search(rb'\x66\x8b\x04\xfd(?P<indexes_addr>[\s\S]{4})', pe.get_memory_mapped_image())
indexes_addr = struct.unpack("I", match.group("indexes_addr"))[0]

for i in range(25):
    index_addr = indexes_addr + i * 8
    index_rva = index_addr - pe.OPTIONAL_HEADER.ImageBase
    enc_string_index = pe.get_word_at_rva(index_rva)
    
    plain_string = decrypt_string(pe, enc_string_index)
    print(f"{enc_string_index}: {plain_string}")

172: os_shutdown
173: os_reboot
174: bot_uninstall
175: bot_update
176: bot_bc_add
177: bot_bc_remove
178: bot_httpinject_disable
179: bot_httpinject_enable
180: fs_path_get
181: fs_search_add
182: fs_search_remove
183: user_destroy
184: user_logoff
185: user_execute
186: user_cookies_get
187: user_cookies_remove
188: user_certs_get
189: user_certs_remove
190: user_url_block
191: user_url_unblock
192: user_homepage_set
193: user_ftpclients_get
194: user_emailclients_get
195: user_flashplayer_get
196: user_flashplayer_remove
